In [ ]:
from typing import Callable

import matplotlib.pyplot as plt
import numpy as np

plt.style.use("ggplot")

In [ ]:
def explicit_euler(f_prime: Callable, x: np.ndarray, y0: float) -> np.ndarray:
    y = np.zeros(x.shape)
    y[0] = y0

    for i in range(len(x) - 1):
        h = x[i + 1] - x[i]
        k1 = f_prime(x[i], y[i])
        y[i + 1] = y[i] + h * k1

    return y

# AKA:  Heun
def enhanced_euler(f_prime: Callable, x: np.ndarray, y0: float) -> np.ndarray:
    y = np.zeros(x.shape)
    y[0] = y0

    for i in range(len(x) - 1):
        h = x[i + 1] - x[i]
        k1 = f_prime(x[i], y[i])
        k2 = f_prime(x[i] + h, y[i] + h * k1)
        y[i + 1] = y[i] + h * (k1 + k2) / 2.0

    return y

def modified_euler(f_prime: Callable, x: np.ndarray, y0: float) -> np.ndarray:
    y = np.zeros(x.shape)
    y[0] = y0

    for i in range(len(x) - 1):
        h = x[i + 1] - x[i]
        k1 = f_prime(x[i], y[i])
        k2 = f_prime(x[i] + h / 2.0, y[i] + h * k1 / 2.0)
        y[i + 1] = y[i] + h * k2

    return y

def runge_kutta_3(f_prime: Callable, x: np.ndarray, y0: float) -> np.ndarray:
    y = np.zeros(x.shape)
    y[0] = y0

    for i in range(len(x) - 1):
        h = x[i + 1] - x[i]
        yi = y[i]
        k1 = f_prime(x[i], y[i])
        k2 = f_prime(x[i] + 0.50 * h, y[i] + 0.50 * k1 * h)
        k3 = f_prime(x[i] + 0.75 * h, y[i] + 0.75 * k2 * h)
        y[i + 1] = yi + h * (2 * k1 + 3 * k2 + 4 * k3) / 9.0

    return y

def runge_kutta_4(f_prime: Callable, x: np.ndarray, y0: float) -> np.ndarray:
    y = np.zeros(x.shape)
    y[0] = y0

    for i in range(len(x) - 1):
        h = x[i + 1] - x[i]
        yi = y[i]
        k1 = f_prime(x[i], yi)
        k2 = f_prime(x[i] + 0.5 * h, yi + 0.5 * k1 * h)
        k3 = f_prime(x[i] + 0.5 * h, yi + 0.5 * k2 * h)
        k4 = f_prime(x[i] + 1.0 * h, yi + 1.0 * k3 * h)
        y[i + 1] = yi + h * (k1 + 2.0 * k2 + 2.0 * k3 + k4) / 6.0

    return y


def local_error(analytical: np.ndarray, numerical: np.ndarray) -> np.ndarray:
    return np.abs(analytical - numerical)


def global_error(analytical: np.ndarray, numerical: np.ndarray) -> np.ndarray:
    return np.abs(analytical - numerical) / analytical

### Exercício A


In [ ]:
def calculate_approximations(f_prime: Callable, analytical_f: Callable, a: float, b: float, y0: float, h_values: np.ndarray):
    local_errors = {}
    global_errors = {}

    plt.figure(figsize=(10, 8))
    for h in h_values:
        n = int((b - a) / h)
        x = np.linspace(a, b, n + 1)

        y = explicit_euler(f_prime, x, y0)
        plt.plot(x, y, label=f"Explicit Euler, h = {h}")
        local_errors.update({f"Explicit Euler, h = {h}": local_error(analytical_f(x), y)})
        global_errors.update({f"Explicit Euler, h = {h}": global_error(analytical_f(x), y)})

        y = enhanced_euler(f_prime, x, y0)
        plt.plot(x, y, label=f"Enhanced Euler, h = {h}", linestyle="dotted")
        local_errors.update({f"Enhanced Euler, h = {h}": local_error(analytical_f(x), y)})
        global_errors.update({f"Enhanced Euler, h = {h}": global_error(analytical_f(x), y)})

        y = modified_euler(f_prime, x, y0)
        plt.plot(x, y, label=f"Modified Euler, h = {h}", linestyle="dashdot")
        local_errors.update({f"Modified Euler, h = {h}": local_error(analytical_f(x), y)})
        global_errors.update({f"Modified Euler, h = {h}": global_error(analytical_f(x), y)})

        y = runge_kutta_3(f_prime, x, y0)
        plt.plot(x, y, label=f"Runge-Kutta 3, h = {h}", linestyle="dashed")
        local_errors.update({f"Runge-Kutta 3, h = {h}": local_error(analytical_f(x), y)})
        global_errors.update({f"Runge-Kutta 3, h = {h}": global_error(analytical_f(x), y)})

        y = runge_kutta_4(f_prime, x, y0)
        plt.plot(x, y, label=f"Runge-Kutta 4, h = {h}", linestyle="dashdot")
        local_errors.update({f"Runge-Kutta 4, h = {h}": local_error(analytical_f(x), y)})
        global_errors.update({f"Runge-Kutta 4, h = {h}": global_error(analytical_f(x), y)})

        
    plt.plot(x, analytical_f(x), label="Analítica", color="black", linewidth=2, linestyle="dashed")
    plt.legend()
    plt.title("Comparação entre métodos numéricos e analítico")
    plt.show()


    for method, errors in local_errors.items():
        x = np.linspace(a, b, len(errors))
        plt.plot(x, errors, label=method)

    plt.legend()
    plt.title("Erro local")
    plt.show()

    for method, errors in global_errors.items():
        x = np.linspace(a, b, len(errors))
        plt.plot(x, errors, label=method)

    plt.legend()
    plt.title("Erro global")
    plt.show()

In [ ]:
f_prime = lambda x, y: np.exp(-x) - 2 * y
analytical_f = lambda x: np.exp(-x) + 2 * np.exp(-2 * x)

a, b = 0, 10
y0 = 3
h_values = [0.5]

calculate_approximations(f_prime, analytical_f, a, b, y0, h_values)


In [ ]:
f_prime = lambda x, y: x - 2  * y + 1
y0 = 1

a, b = 0, 1
h_values = [0.1]

analytical_f = lambda x: (3 * np.exp(-2 * x) + 2 * x + 1) / 4

calculate_approximations(f_prime, analytical_f, a, b, y0, h_values)
